In [10]:
# Puzzle 1
def get_data(filename):
    with open(filename, "r") as file:
        d_workflows = {}
        for line in file:
            # Check, if there are still workflows
            if line == "\n":
                break
            line = line.strip()
            wf_name, wf_cont = line.split("{")
            wf_cont = wf_cont[:-1]
            wf_cont = wf_cont.split(",")
            l_rules = []
            for rule in wf_cont:
                if ":" in rule:
                    cond, act = rule.split(":")
                else:
                    cond, act = "no_cond", rule
                l_rules.append((cond, act))
            d_workflows[wf_name] = l_rules
        
        # Now the parts are taken
        l_parts = []
        for line in file:
            line = line.strip()
            line = line[1:-1]
            line = line.split(",")
            x = int(line[0].split("=")[1]) 
            m = int(line[1].split("=")[1])
            a = int(line[2].split("=")[1]) 
            s = int(line[3].split("=")[1]) 
            part = {"x" : x, "m" : m, "a" : a, "s": s}
            l_parts.append(part)
    return d_workflows, l_parts

def check_rule(rule, part):
    cond, act = rule
    # Check if rule has no condition
    if cond == "no_cond":
        return act
    elif "<" in cond:
            var, val = cond.split("<")
            val = int(val)
            if part[var] < val:
                return act
            else:
                return "FAIL"
    elif ">" in cond:
            var, val = cond.split(">")
            val = int(val)
            if part[var] > val:
                return act
            else:
                return "FAIL"
    else:
        print("Error: Could not handle rule: ", rule, part)

def check_wf(wf, part):
    global d_workflows
    l_rules = d_workflows[wf]
    for rule in l_rules:
        result = check_rule(rule, part)
        if not (result == "FAIL"):
            return result

def check_part(part):
    next_wf = "in"
    while True:
        next_wf = check_wf(next_wf, part)
        if next_wf == "A":
            return "A"
        if next_wf == "R":
            return "R"
        
def check_all_parts(l_parts):
    l_accepted = []
    for part in l_parts:
        if check_part(part) == "A":
            l_accepted.append(part)
    return l_accepted

def calc_total(l_accepted):
    total = 0
    for part in l_accepted:
        for cat in part:
            total += part[cat]
    return total
      
        
# Main Program Puzzle 1
d_workflows, l_parts = get_data("Test.txt")    
l_accepted = check_all_parts(l_parts)
total = calc_total(l_accepted)
print("Puzzle 1: The total of all accepted parts is:", total)
print(d_workflows)

Puzzle 1: The total of all accepted parts is: 19114
{'px': [('a<2006', 'qkq'), ('m>2090', 'A'), ('no_cond', 'rfg')], 'pv': [('a>1716', 'R'), ('no_cond', 'A')], 'lnx': [('m>1548', 'A'), ('no_cond', 'A')], 'rfg': [('s<537', 'gd'), ('x>2440', 'R'), ('no_cond', 'A')], 'qs': [('s>3448', 'A'), ('no_cond', 'lnx')], 'qkq': [('x<1416', 'A'), ('no_cond', 'crn')], 'crn': [('x>2662', 'A'), ('no_cond', 'R')], 'in': [('s<1351', 'px'), ('no_cond', 'qqz')], 'qqz': [('s>2770', 'qs'), ('m<1801', 'hdj'), ('no_cond', 'R')], 'gd': [('a>3333', 'R'), ('no_cond', 'R')], 'hdj': [('m>838', 'A'), ('no_cond', 'pv')]}


In [5]:
possibilities = {"x" : (1, 4000), "m" : (1, 4000), "a" : (1, 4000), "s" : (1, 4000)}
rule = ("s>3448", "A")

In [42]:
# Puzzle 2
def get_data(filename):
    with open(filename, "r") as file:
        d_workflows = {}
        for line in file:
            # Check, if there are still workflows
            if line == "\n":
                break
            line = line.strip()
            wf_name, wf_cont = line.split("{")
            wf_cont = wf_cont[:-1]
            wf_cont = wf_cont.split(",")
            l_rules = []
            for rule in wf_cont:
                if ":" in rule:
                    cond, act = rule.split(":")
                else:
                    cond, act = "no_cond", rule
                l_rules.append((cond, act))
            d_workflows[wf_name] = l_rules
    return d_workflows

def calc_possibilities(possibilities):
    poss = 1
    for char in possibilities:
        x, y = possibilities[char]
        poss *= y - x + 1
    return poss

def split_interval(interval, value):
    if interval[0] <= value <= interval[1]:
        return (interval[0], value - 1), (value, interval[1])
    elif value > interval[1]:
        return (interval, (value, -1))
    elif value < interval[0]:
        return ((value, -1), interval)
    else:
        print("Error, cannot split:", interval, value)
        
def get_possibilities(possibilities, wf_name, rule_no):
    #print(possibilities)
    #print(wf_name, rule_no)
    global d_workflows
    rule = d_workflows[wf_name][rule_no]
    cond, act = rule
    if cond == "no_cond":
        if act == "R":
            return 0
        elif act == "A":
            return calc_possibilities(possibilities)
        else:
            return get_possibilities(possibilities, act, 0)
    else:
        if "<" in cond:
            var, val = cond.split("<")
            val = int(val)
            intv1, intv2 = split_interval(possibilities[var], val)
            poss_intv1 = possibilities.copy()
            poss_intv1[var] = intv1
            poss_intv2 = possibilities.copy()
            poss_intv2[var] = intv2
            if act == "R":
                poss = 0
            elif act == "A":
                poss = calc_possibilities(poss_intv1)
            else:
                poss = get_possibilities(poss_intv1, act, 0)
            poss += get_possibilities(poss_intv2, wf_name, rule_no + 1)
            return poss
        elif ">" in cond:
            var, val = cond.split(">")
            val = int(val)
            intv1, intv2 = split_interval(possibilities[var], val + 1)
            poss_intv1 = possibilities.copy()
            poss_intv1[var] = intv1
            poss_intv2 = possibilities.copy()
            poss_intv2[var] = intv2
            if act == "R":
                poss = 0
            elif act == "A":
                poss = calc_possibilities(poss_intv2)
            else:
                poss = get_possibilities(poss_intv2, act, 0)
            poss += get_possibilities(poss_intv1, wf_name, rule_no + 1)
            return poss
        else:
            print("Error: Rule does neither contain < or >")
            
    
        

        

    
# Mainprogram Puzzle 2
possibilities = {"x" : (1, 4000), "m" : (1, 4000), "a" : (1, 4000), "s" : (1, 4000)}
        
d_workflows = get_data("Puzzle.txt")          
print(get_possibilities(possibilities, "in", 0))
        
      

127675188176682
